In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install lion-pytorch

In [4]:
%pip install torchmetrics

In [5]:
import torch

from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from lion_pytorch import Lion
from utility.step_lr import StepLR


In [6]:

def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        optimizer.zero_grad()

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions, targets, smoothing=0.1)
        loss.mean().backward()

        optimizer.step()

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            scheduler(epoch)

In [7]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [8]:
if __name__ == "__main__":

    num_epochs = 50
    model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    model_name = 'widerestnet-sgd'
    model_name = model_name_path + model_name
    initialize(seed=42)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = WideResNet(16, 8, dropout=0.0, in_channels=3, labels=2).to(device)

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=32)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.002, weight_decay=0.01)
    scheduler = StepLR(optimizer, 0.005, num_epochs)

    for epoch in range(num_epochs):
        train(model, device, dataset.train, optimizer, log, scheduler)
        test(model, device, dataset.valid, log)

    log.flush()
    # ┃          49  ┃      0.1876  │      0.86 %  ┃   4.000e-05  │       00:02  ┃      0.3099  │     74.00 %  ┃

    # torch.save(model.state_dict(), '../model_trained/'+model_name+'.pth')

┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      0.3620  │      0.53 %  ┃   5.000e-03  │       00:10  ┃      0.3685  │     50.00 %  ┃
┃           1  ┃      0.3503  │      0.52 %  ┃   5.000e-03  │       00:02  ┃      0.3655  │     59.33 %  ┃
┃           2  ┃      0.3383  │      0.71 %  ┃   5.000e-03  │       00:02  ┃      0.3636  │     65.33 %  ┃
┃           3  ┃      0.3288  │      0.72 %  ┃   5.000e-03  │       00:02  ┃      0.3632  │     65.33 %  ┃
┃           4  ┃      0.3187  │      0.77 %  ┃   5.000e-03  │       00:02  ┃      0.3639  │     61.00 %  ┃
┃           5  ┃      0.3063  │      0.79 %  ┃   5.000e-03  │       00:02  ┃      0.3641  │     62.00 %  ┃
┃           6  ┃      0.2961  │      0.80 %  ┃   5.000e-03  │       00:02  ┃      0.3642  │     60.67 %  ┃
┃           7  ┃      0.2869  │      0.81 %  ┃   5.000e-03  │       00:02  ┃      0.3613  │     62.67 %  ┃
┃           8  ┃      0.2760  │      0.82 %  ┃   5.000e-03  │       00:02  ┃      0.3576  │     63.33 %  ┃
┃           9  ┃      0.2663  │      

In [9]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [10]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)


tensor(0.9346, device='cuda:0')
tensor(0.9300, device='cuda:0')
tensor(0.8772, device='cuda:0')
tensor(0.8600, device='cuda:0')
tensor(1., device='cuda:0')
